In [4]:
from collections import defaultdict
from pathlib import Path
import shutil
import re

import numpy as np

data_path = Path("/home/aleksandernagaj/Milestone/data/RPIfield_v0")
exported_path = data_path / "exported"
exported_path.mkdir(exist_ok=True)

appearances = defaultdict(int)
for cam_dir in (data_path / "Data").iterdir():
    for person_dir in cam_dir.iterdir():
        person_id = person_dir.name.split("_")[0]
        appearances[person_id] += 1

ids: np.ndarray = np.array(list(appearances.keys()), dtype=int)
counts: np.ndarray = np.array(list(appearances.values()), dtype=int)

distractors: np.ndarray = ids[counts == 1]
reappearing: np.ndarray = ids[counts > 1]

# split reappearing ids into train and test with 50/50 ratio
# np.random.seed(42)
# np.random.shuffle(reappearing)
# train_ids = reappearing[: len(reappearing) // 2]
# test_ids = reappearing[len(reappearing) // 2 :]

for cam_dir in (data_path / "Data").iterdir():
    cam_label: int = int(cam_dir.name.split("_")[1])
    for person_dir in cam_dir.iterdir():
        person_label = person_dir.name.split("_")
        person_id = int(person_label[0])

        if person_id in reappearing:
            if len(person_label) > 1:
                # treat every reappearance as a new person
                person_id += 900

            for image_path in person_dir.iterdir():
                shutil.copy(
                    image_path,
                    exported_path
                    / f"{person_id:0>4d}_c{cam_label:0>2d}_{image_path.name.split('_')[-1].split('.')[0]:0>6}.{image_path.name.split('.')[-1]}",
                )

['97', '1']
['76', '2']
['27', '1']
['24', '1']
['1', '2']
['99', '2']
['67', '2']
['45', '1']
['67', '1']
['72', '1']
['44', '1']
['76', '1']
['96', '1']
['112', '2']
['19', '2']
['22', '2']
['112', '1']
['54', '1']
['51', '1']
['110', '1']
['99', '1']
['13174', '1']
['21', '1']
['21', '2']
['54', '2']
['98', '2']
['96', '2']
['16', '3']
['86', '1']
['35', '1']
['98', '1']
['92', '2']
['34', '1']
['68', '1']
['15', '2']
['38', '1']
['80', '1']
['18', '2']
['92', '1']
['95', '1']
['64', '1']
['1', '1']
['19', '1']
['49', '1']
['24', '2']
['16', '4']
['88', '1']
['22', '1']
['96', '3']
['111', '1']
['112', '3']
['5', '1']
['16', '2']
['15', '1']
['95', '2']
['6', '1']
['16', '1']
['33', '1']
['47', '1']
['90', '1']
['97', '2']
['17', '1']
['4', '1']
['4', '2']
['18', '4']
['85', '2']
['42', '1']
['108', '1']
['18', '1']
['18', '3']
['85', '1']
['1', '3']
['53', '1']
['2', '1']
['59', '1']
['62', '1']
['57', '2']
['68', '1']
['31', '1']
['66', '1']
['57', '1']
['14', '1']
['2', '2']
['9'

In [5]:
(data_path / "exported_1fps").mkdir(exist_ok=True)

# downsample frame rate to 1 frame per second in exported dataset
for image_path in exported_path.iterdir():
    sequence_no = int(image_path.name.split("_")[-1].split(".")[0])
    if sequence_no % 7 == 0:
        shutil.copy(
            image_path,
            data_path / "exported_1fps" / image_path.name,
        )

In [10]:
from scipy.io import loadmat
annotations = []
for info in (data_path / "Data_Info").iterdir():
    if info.name.endswith(".mat"):
        annotations.append(loadmat(info))
print(annotations[0]["allimagenames"])

[[array(['1\\Cam_9f_44619.png'], dtype='<U18')
  array(['1\\Cam_9f_44620.png'], dtype='<U18')
  array(['1\\Cam_9f_44621.png'], dtype='<U18') ...
  array(['15106\\Cam_9f_47940.png'], dtype='<U22')
  array(['15107\\Cam_9f_47445.png'], dtype='<U22')
  array(['15107\\Cam_9f_47446.png'], dtype='<U22')]]


In [21]:
from collections import defaultdict
import re
import cv2
from pathlib import Path
import numpy as np

data_path = Path("/home/aleksandernagaj/Milestone/data/RPIfield_v0")
imag_paths = list(sorted((data_path / "reid_video" / "train" / "0006").iterdir()))
# print(imag_paths)
dxs, dys = [], []
direction_by_camera = defaultdict(list)

for i in range(1, len(imag_paths)):
    camera_id1 = re.search(r"c(\d+)", imag_paths[i-1].name).group(1)
    camera_id2 = re.search(r"c(\d+)", imag_paths[i].name).group(1)
    if camera_id1 != camera_id2:
        continue

    img1 = cv2.imread(str(imag_paths[i - 1]), 0)
    img2 = cv2.imread(str(imag_paths[i]), 0)
    # Initialize the ORB detector
    sift = cv2.SIFT_create()

    # Find keypoints and descriptors
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)

    # Create a BFMatcher object
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)

    # Match the descriptors
    matches = bf.match(des1, des2)

    # Calculate average shift
    dx, dy = 0, 0
    for match in matches:
        img1_idx = match.queryIdx
        img2_idx = match.trainIdx
        (x1, y1) = kp1[img1_idx].pt
        (x2, y2) = kp2[img2_idx].pt
        dx += (x2 - x1)
        dy += (y2 - y1)

    dx /= len(matches)
    dy /= len(matches)
    # print(dx, dy)

    direction_by_camera[camera_id1].append((dx, dy))

for cam, directions in direction_by_camera.items():
    dxs, dys = zip(*directions)
    print(cam, np.mean(dxs), np.mean(dys))

02 -0.2849571245922986 -2.50735840766071
03 0.03438546181837095 0.5875314022478545
04 0.1367734670135597 0.43803174049517474
05 -0.4732053908708432 1.2583781274338406
06 -0.12198161387396905 0.5566333270507852
07 -0.18343296105712228 0.23851913692050467
08 -0.12690057332853655 0.29103370294875924
09 1.6168286836482082 1.262650955291022
10 -0.726592348262659 -0.9937912999527196
11 -0.782810092600613 -0.9860660891710755
12 0.9591137828401343 -0.2884478651505569
